# package

In [3]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../'
sys.path.append(base_dir)
import os
from utils import *
import os
import polars as pl
import numpy as np
import pandas as pd
from functools import lru_cache
from utils import *

# Config 

In [4]:
task = 'task3'

# Read data 

In [5]:
# ls ../data/sub_files/ | grep test4task3

In [6]:
# task2 = pl.scan_parquet('../data/sub_files/task2_test4task3_task2_rank_lgbm_v2.parquet')
# task1 = pl.scan_parquet('../data/sub_files/task1_test4task3_task1_rank_lgbm_v10.parquet')


test4task3_pl = pl.scan_parquet(os.path.join('../data/raw_data_session_id/', 'sessions_test_task3.parquet')).with_columns(pl.col('prev_items').apply(str2list))


In [7]:
test4task3_pl.collect().shape

(56422, 3)

In [11]:
test4task3_pl.head().collect()

prev_items,locale,session_id
list[str],str,i64
"[""B0BF9JMVDG"", ""B01ET9V90M""]","""ES""",4365996
"[""B09QQG85HM"", ""B09J4T4JF5""]","""ES""",4365997
"[""B09NSKDG4K"", ""B09YY6J1ZM""]","""ES""",4365998
"[""B09B7NYDJ7"", ""B09B7NYDJ7""]","""ES""",4365999
"[""B0B6J17LK4"", ""B0B6R7X6GY"", ""B07HXY5SGH""]","""ES""",4366000


In [10]:
# task1.head().collect()

In [12]:
# res = pl.concat([task1, task2], how='vertical')
product_pl = (pl.scan_parquet(os.path.join('../data/raw_data_session_id/', 
                                          'products_train.parquet'))
                  # .with_columns(
                  #     pl.when(pl.col('locale')=='DE').then(1).when(pl.col('locale')=='DE')
                  #       .then(2)
                  #       .otherwise(3).alias('locale')
                  # )
             )

In [13]:
# product_pl.select(['id', 'locale', 'title']).head().collect()

In [14]:
# res.head().collect()

In [15]:
# test4task3_pl.schema

In [17]:
predictions_step1 = (
    test4task3_pl
    .with_columns(
        pl.col('prev_items').arr.get(-1).alias('next_item_prediction')
    )
    .with_columns(
        pl.when(pl.col('next_item_prediction').is_null()).then(pl.col('prev_items').arr.get(-1)).otherwise(pl.col('next_item_prediction'))
        .alias('next_item_prediction')
    )
)#.head().collect()

In [26]:
predictions_step1.head().collect()

prev_items,locale,session_id,next_item_prediction
list[str],str,i64,str
"[""B0BF9JMVDG"", ""B01ET9V90M""]","""ES""",4365996,"""B01ET9V90M"""
"[""B09QQG85HM"", ""B09J4T4JF5""]","""ES""",4365997,"""B09J4T4JF5"""
"[""B09NSKDG4K"", ""B09YY6J1ZM""]","""ES""",4365998,"""B09YY6J1ZM"""
"[""B09B7NYDJ7"", ""B09B7NYDJ7""]","""ES""",4365999,"""B09B7NYDJ7"""
"[""B0B6J17LK4"", ""B0B6R7X6GY"", ""B07HXY5SGH""]","""ES""",4366000,"""B07HXY5SGH"""


In [19]:
predictions = (
    predictions_step1
        .join(product_pl.select(['id', 'locale', 'title']), how='left', right_on=['id', 'locale']
                 , left_on=['next_item_prediction', 'locale'])
        .with_columns(pl.col('title').alias('next_item_prediction').cast(pl.Utf8))
)#.head().collect()

In [20]:
assert predictions.filter(pl.col('title').is_null()).collect().shape[0] == 0

In [21]:
assert test4task3_pl.collect().shape[0] == predictions.collect().shape[0]

In [22]:
# product_pl.filter(pl.col('id')=='B085NXQL7F').collect()

In [23]:
# test4task3_pl.filter(pl.col('session_id')==4420531).collect()

In [27]:
predictions.head().collect()

prev_items,locale,session_id,next_item_prediction,title
list[str],str,i64,str,str
"[""B0BF9JMVDG"", ""B01ET9V90M""]","""ES""",4365996,"""Pata Negra Rob…","""Pata Negra Rob…"
"[""B09QQG85HM"", ""B09J4T4JF5""]","""ES""",4365997,"""Xiaomi REDMI T…","""Xiaomi REDMI T…"
"[""B09NSKDG4K"", ""B09YY6J1ZM""]","""ES""",4365998,"""Correa Xiaomi …","""Correa Xiaomi …"
"[""B09B7NYDJ7"", ""B09B7NYDJ7""]","""ES""",4365999,"""Rowenta Mini E…","""Rowenta Mini E…"
"[""B0B6J17LK4"", ""B0B6R7X6GY"", ""B07HXY5SGH""]","""ES""",4366000,"""Fiambrera Eléc…","""Fiambrera Eléc…"


In [28]:
predictions.filter(pl.col('title').is_null()).collect()

prev_items,locale,session_id,next_item_prediction,title
list[str],str,i64,str,str


In [29]:

# predictions.select('locale').collect().to_series().value_counts()

In [30]:
predictions_df = predictions.collect().to_pandas()

In [31]:
predictions_df['next_item_prediction'] = predictions_df['next_item_prediction'].astype('str')

# Validate predictions ✅

In [32]:
def check_predictions(predictions, test_sessions):
    """
    These tests need to pass as they will also be applied on the evaluator
    """
    test_locale_names = test_sessions['locale'].unique()
    for locale in test_locale_names:
        sess_test = test_sessions.query(f'locale == "{locale}"')
        preds_locale =  predictions[predictions['locale'] == sess_test['locale'].iloc[0]]
        assert sorted(preds_locale.index.values) == sorted(sess_test.index.values), f"Session ids of {locale} doesn't match"
        assert predictions['next_item_prediction'].apply(lambda x: isinstance(x, str)).all(), "Predictions should all be strings"

In [33]:
predictions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56422 entries, 0 to 56421
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   prev_items            56422 non-null  object
 1   locale                56422 non-null  object
 2   session_id            56422 non-null  int64 
 3   next_item_prediction  56422 non-null  object
 4   title                 56422 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.2+ MB


In [34]:
check_predictions(predictions_df, test_sessions=test4task3_pl.collect().to_pandas())

In [35]:
sub_file = f'submission_{task}.parquet'

In [36]:
! ls -al {sub_file}

-rw-r--r-- 1 jupyter jupyter 12565524 Jun  8 19:50 submission_task3.parquet


In [37]:
# Its important that the parquet file you submit is saved with pyarrow backend
predictions_df.to_parquet(sub_file, engine='pyarrow')

In [38]:
f'submission_{task}.parquet'

'submission_task3.parquet'

## Submit to AIcrowd 🚀

In [145]:
# You can submit with aicrowd-cli, or upload manually on the challenge page.
# !aicrowd submission create -c task-3-next-product-title-generation -f {sub_file}